### Brouillon

In [1]:
import numpy as np
import pylab
import healpy as hp
import pysm3 
import pysm3.units as u
import matplotlib.pyplot as py
import cmbdb as cmb
from fgbuster import algebra, observation_helpers,cosmology
from fgbuster.mixingmatrix import MixingMatrix

In [2]:
from fgbuster.component_model import CMB, Dust, Synchrotron

In [3]:
NSIDE = 64
lmax = 3*NSIDE-1
print(lmax)
amin2rad = np.pi/(60.*180.)
print(amin2rad)

191
0.0002908882086657216


### Preliminaries slicing functions

In [4]:
def get_last2d(data):
            if data.ndim <= 2:
                return data
            slc = [0] * (data.ndim - 2)
            slc += [slice(None), slice(None)]
            return slc
            # return data[slc]

In [5]:
def get_last_2d_V2(x):
    m,n = x.shape[-2:]
    return x.flat[:m*n].reshape(m,n)

In [6]:
data_test = np.zeros((5,6,7,8,9,10))

In [7]:
print(get_last_2d_V2(data_test).shape)
# print(type(get_last2d(data_test)[4]))

(9, 10)


In [8]:
print(slice(None))

slice(None, None, None)


In [9]:

#Added by Clement Leloup
def harmonic_noise_cov(instrument, lmax, bl=None):

    if bl is None:
        try:
            bl = np.array([hp.gauss_beam(np.radians(b/60.), lmax=lmax)
                           for b in instrument.fwhm])
        except AttributeError:
            bl = np.ones((len(instrument.frequency), lmax+1))
        bl = np.repeat(bl[:,np.newaxis,:], 3, axis=1)

    nl = (np.array(bl) / np.radians(instrument.depth_p/60.)[:,np.newaxis,np.newaxis])**2

    return nl

In [10]:
instrument = observation_helpers.get_instrument('LiteBIRD')
fwhm = instrument.fwhm*amin2rad
freq = instrument.frequency
fwhm_common = 80*amin2rad
print(fwhm.shape)
print(freq)


(15,)
0      40.0
1      50.0
2      60.0
3      68.0
4      78.0
5      89.0
6     100.0
7     119.0
8     140.0
9     166.0
10    195.0
11    235.0
12    280.0
13    337.0
14    402.0
Name: frequency, dtype: float64


In [11]:
inv_noise =cosmology.harmonic_noise_cov(instrument=instrument,lmax=lmax)
print(inv_noise.shape)

(15, 3, 192)


/home/julientang2001/fgbuster/fgbuster/cosmology.py:759: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  nl = (np.array(bl) / np.radians(instrument.depth_p/60.)[:,np.newaxis,np.newaxis])**2


### Create maps

In [12]:

# sky_CMB = get_sky(NSIDE,'c1')
sky_d0s0 = observation_helpers.get_sky(NSIDE,'d0s0') 
# freq_maps_CMB = get_observation(instrument, sky_CMB)
freq_maps_d0s0 = observation_helpers.get_observation(instrument,sky_d0s0)
num_freq,ncomp,npix = freq_maps_d0s0.shape
print(freq_maps_d0s0.shape)
alms_fgs = hp.map2alm(freq_maps_d0s0[0],lmax =lmax) [0]

print(alms_fgs.shape)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


(15, 3, 49152)
(18528,)


In [13]:
ell_em = hp.Alm.getlm(lmax, np.arange(alms_fgs.shape[-1]))[0]
ell_em = np.stack((ell_em, ell_em), axis=-1).reshape(-1) # For transformation into real alms

    # Transform healpy complex alms into real alms
# alms_fgs = cosmology._format_alms(alms_fgs, lmin=0)

    # Format the estimated inverse noise matrix
invNl = cosmology.harmonic_noise_cov(instrument, lmax)
invNl = np.array([[np.diag(invNl[:,st,l]) for st in np.arange(invNl.shape[1])] for l in np.arange(invNl.shape[2])])
invNlm = np.array([invNl[l,1:,:,:] for l in ell_em]) # Here we take only polarization
invNl = invNl[:,-1,:,:]


/home/julientang2001/fgbuster/fgbuster/cosmology.py:759: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  nl = (np.array(bl) / np.radians(instrument.depth_p/60.)[:,np.newaxis,np.newaxis])**2


In [14]:
print(invNl.shape)

(192, 15, 15)


# Beam

In [27]:
beam_array = np.array([hp.gauss_beam(f,lmax = lmax, pol=True) for f in fwhm])
print(beam_array.shape)
beam_array = beam_array[:,:,1:3]
print(beam_array.shape)


(15, 192, 4)
(15, 192, 2)


In [29]:

num_freq, num_l, num_stokes = beam_array.shape
beam_array_lm = beam_array[:,ell_em,:]
beam_diag_lm = np.array([[np.diag(beam_array_lm[:,i,j]) for i in range(0,np.size(ell_em))]for j in range(0,num_stokes)])
beam_diag_lm = np.transpose(beam_diag_lm, (1, 0, 2, 3))
print(beam_diag_lm.shape)

(37056, 2, 15, 15)


In [16]:
print(ell_em)

[  0   0   1 ... 191 191 191]


# Mixing matrix with beam

In [17]:
components = [CMB(), Dust(353.), Synchrotron(23.)]
# The starting point of the fit is the ax1sm default value, so let's shift it
components[1].defaults = [1.54, 20.]
components[2].defaults = [-3.0]
A = MixingMatrix(*components)
A_ev= A.evaluator(instrument.frequency)
i_cmb = A.components.index('CMB')

In [18]:
A_ev_beam = A.evaluator_beam(instrument.frequency,beam_diag_lm)
print(type(A_ev_beam))

<class 'function'>


In [19]:
x0 = np.array([x for c in components for x in c.defaults])
A_ev_beam_x0 = A_ev_beam(x0)
print(A_ev_beam_x0.shape)
# A_ev_x0 = A.evaluator(instrument.frequency)(x0)
# print(A_ev_x0.shape)

(4, 37056, 15, 3)


In [20]:
A_ev_db1 = A.diff(instrument.frequency,*x0)
print(len(A_ev_db1))
A_ev_db1_arr = np.array(A_ev_db1)
print(A_ev_db1_arr.shape)

3
(3, 15, 1)


/home/julientang2001/fgbuster/fgbuster/mixingmatrix.py:187: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  res += [g[..., np.newaxis]


In [21]:
A_dB_x0 = (A.diff_evaluator(instrument.frequency))(x0)

print(type(A_dB_x0))
print(len(A_dB_x0))
print(A_dB_x0[0])

<class 'list'>
3
[[-0.00921547]
 [-0.01178951]
 [-0.01437917]
 [-0.01646693]
 [-0.01910662]
 [-0.02206425]
 [-0.0250968 ]
 [-0.03056291]
 [-0.03701476]
 [-0.04565944]
 [-0.05600231]
 [-0.06995021]
 [-0.07783224]
 [-0.03649189]
 [ 0.27183877]]


In [22]:
print(len(x0))

3


In [25]:
A_ev_dB = A.diff_beam(instrument.frequency,beam_diag_lm,*x0)
print(type(A_ev_dB))
print(A_ev_dB[0].shape)

<class 'list'>
(4, 37056, 15, 1)


/home/julientang2001/fgbuster/fgbuster/mixingmatrix.py:198: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  res += [g[..., np.newaxis]


In [23]:
A_ev_ddb1 = A.diff_diff(instrument.frequency,*x0)
print(len(A_ev_ddb1))
A_ev_ddb1_arr = np.array(A_ev_ddb1)
print(A_ev_ddb1_arr.shape)

AttributeError: 'Series' object has no attribute 'reshape'

In [22]:
A_ev_ddB = A.diff_diff_beam(instrument.frequency,beam_diag_lm,*x0)
print(type(A_ev_ddB))

AttributeError: 'Series' object has no attribute 'reshape'

In [ ]:
likelihood = algebra._build_bound_inv_logL_and_logL_dB(A_ev_beam,alms_fgs,invN=inv_noise)

In [ ]:
algebra.comp_sep(A_ev_beam,alms_fgs,invNl,None,None,*x0)

In [ ]:
print(type(likelihood))

In [ ]:
print(type(likelihood[0]))

In [ ]:
likelihood[0](np.array([1,2,3]))